## Portfolio Optimization using Julia
Typical portfolio optimization problems focus on maximizing expected returns. The techniques that are used are typically linear programming, mixed integer programming, among others. 

This is a simple example of how linear programming can be applied to a portfolio optimization problem where the focus, or objective, is to maximize monthly dividend income...

An investor has only so much money to invest in a portfolio of different dividend stocks/ETFs. How does he/she choose the optimal amount of each stock/ETF in such a way to maximize his/her dividend income, while making sure a diversified portfolio is maintained?

This is a problem that can be solved by linear optimization.

In this case, our unknowns (variables) are how many shares of each stock to hold, and our constraints are 1) the amount available to invest, and 2) the portfolio should be diversified to a certain extent.

Let's say we want to invest in 4 stocks only, and their prices and dividend yields are as follows:

Ticker Price Dividend Yield
XUT    26.8  0.095
XTR    10.5  0.05
XRE    15.0  0.058
FIE    6.34  0.04

Furthermore, the amount the investor can invest is $75,000, and the investor does not want a single stock to be more than a third of his portfolio.

This all can be formulated as follows.


Maximize:

Total Dividends:
\begin{equation*} \sum_{i \in I} dividend_{i} * shares_{i} \end{equation*}

Subject to:
\begin{equation*} \sum_{i \in I} price_{i} * shares_{i} <= 75,000 \end{equation*}

\begin{equation*} price_{i} * shares_{i} <= 75000 * 0.33 \end{equation*}
\begin{equation*} price_{i} * shares_{i} >= 0.0 \end{equation*}


This can be done easily in Julia using the JuMP and GLPK packages...

In [1]:
using JuMP
using GLPK

In [2]:
m = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [3]:
# tickers: XUT, XTR, XRE, FIE
div = [0.095 0.05 0.058 0.04]
price = [26.8 10.5 15.0 6.34]
limit = 75000

75000

In [4]:
@variable(m, shares[1:4])

4-element Array{VariableRef,1}:
 shares[1]
 shares[2]
 shares[3]
 shares[4]

In [5]:
@objective(m, Max, sum(div[i] * shares[i] for i in 1:4))

0.095 shares[1] + 0.05 shares[2] + 0.058 shares[3] + 0.04 shares[4]

In [6]:
@constraint(m, sum(price[i] * shares[i] for i in 1:4) <= limit)

for i in 1:4
    @constraint(m, price[i] * shares[i] .<= limit * 0.33)
    @constraint(m, price[i] * shares[i] .>= 0.0)
end

In [7]:
println("The optimization problem to be solved is:")
println(m)

The optimization problem to be solved is:
Max 0.095 shares[1] + 0.05 shares[2] + 0.058 shares[3] + 0.04 shares[4]
Subject to
 26.8 shares[1] ≥ 0.0
 10.5 shares[2] ≥ 0.0
 15 shares[3] ≥ 0.0
 6.34 shares[4] ≥ 0.0
 26.8 shares[1] + 10.5 shares[2] + 15 shares[3] + 6.34 shares[4] ≤ 75000.0
 26.8 shares[1] ≤ 24750.0
 10.5 shares[2] ≤ 24750.0
 15 shares[3] ≤ 24750.0
 6.34 shares[4] ≤ 24750.0



In [8]:
JuMP.optimize!(m)
optimvalue = JuMP.objective_value(m)
shares = JuMP.value.(shares)

4-element Array{Float64,1}:
   27.98507462686567
 2357.1428571428573 
 1650.0             
 3903.7854889589908 

In [11]:
println("Total dividends: ", round(optimvalue))
println("Stock 1 (XUT) shares: ", round(shares[1]), "; total \$: ", round(price[1] * shares[1]))
println("Stock 2 (XTR) shares: ", round(shares[2]), "; total \$: ", round(price[2] * shares[2]))
println("Stock 3 (XRE) shares: ", round(shares[3]), "; total \$: ", round(price[3] * shares[3]))
println("Stock 4 (FIE) shares: ", round(shares[4]), "; total \$: ", round(price[4] * shares[4]))

Total dividends: 372.0
Stock 1 (XUT) shares: 28.0; total $: 750.0
Stock 2 (XTR) shares: 2357.0; total $: 24750.0
Stock 3 (XRE) shares: 1650.0; total $: 24750.0
Stock 4 (FIE) shares: 3904.0; total $: 24750.0


So this is saying that dividends would be maximized by investing $24,750 in XTR, XRE, & FIE each, and only $750 in XUT. This is because, for the price of XUT, the dividend yield is much lower relative to the other, cheaper stocks.


This program could be further improved by adding other constrains, such as volatility. For example, if it is important to an investor that the portfolio value does not fluctuate as much, then we can take into account each stock's volatility measure to make sure that we exclude stocks that make the portfolio's volatility greater than a certain level. Various other constraints could be added.